In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Laad een vooraf getraind model (bijv. YOLOv8n)
model = YOLO("yolo12m.pt").to("cuda") 

In [ ]:
# Train het model
# model.train(
#     data=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\YOLOv12v1\data.yaml",       # pad naar je data.yaml
#     epochs=50,              # aantal epochs
#     imgsz=640,              # inputgrootte
#     batch=16,               # pas aan op basis van je GPU/CPU
#     device=0,               # 0 = GPU, "cpu" = CPU
# )

model.train(
    data=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\YOLOv12v1\data.yaml",       # pad naar je data.yaml
    epochs=100,              # aantal epochs
    imgsz=640,              # inputgrootte
    batch=16,               # pas aan op basis van je GPU/CPU
    device=0,
    degrees=30,
    shear=10,
    perspective=0.002,
    flipud=0.1,
    fliplr=0.5,
    mixup=0.2,
    copy_paste=0.2,
    patience=10,
    hsv_h=0.5,
    mosaic=0            # 0 = GPU, "cpu" = CPU
);

# model.tune(
#     data=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\YOLOv12v1\data.yaml",
#     epochs=100,
#     imgsz=640,
#     device=0,   
#     patience=10,
# )

In [ ]:
%matplotlib inline
#  Voorspel (lage confidence threshold)
results = model.predict(source=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\YOLOv12v1\valid\images\IMG_1615_jpg.rf.a86b36c7c0b5e42e968c831cebad7d19.jpg", conf=0.25, save=False, show=False)

# Debug: zijn er boxes?
if results[0].boxes:
    print(f"Gevonden: {len(results[0].boxes)} object(en)")

    # Annotated afbeelding ophalen
    img = results[0].plot()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Laat het zien
    plt.imshow(img)
    plt.axis("off")
    plt.show()
else:
    print("Geen objecten gedetecteerd.")



In [ ]:
model.save("YOLOv12mv8.pt")


In [ ]:
# Pad naar je trainingsresultaten
df = pd.read_csv(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Objectdetectie\runs\detect\train15\results.csv")

# --- Plot 1: Box Loss ---
plt.figure(figsize=(5, 2.5))
plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss')
plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss')
plt.xlabel('Epoch')
plt.ylabel('Box Loss')
plt.title('Box Loss per Epoch')
plt.legend()
plt.show()

# --- Plot 2: Class Loss ---
plt.figure(figsize=(5, 2.5))
plt.plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss')
plt.plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss')
plt.xlabel('Epoch')
plt.ylabel('Class Loss')
plt.title('Class Loss per Epoch')
plt.legend()
plt.show()

# --- Plot 3: DFL Loss ---
plt.figure(figsize=(5, 2.5))
plt.plot(df['epoch'], df['train/dfl_loss'], label='Train DFL Loss')
plt.plot(df['epoch'], df['val/dfl_loss'], label='Val DFL Loss')
plt.xlabel('Epoch')
plt.ylabel('DFL Loss')
plt.title('DFL Loss per Epoch')
plt.legend()
plt.show()

# Plot mAP
plt.figure(figsize=(5, 2.5))
plt.plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@0.5')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('mAP per Epoch val')
plt.legend()
plt.show()


In [ ]:
# Zoek de rij met hoogste mAP@0.5:0.95
best_epoch_row = df.loc[df['metrics/mAP50-95(B)'].idxmax()]

# Print de resultaten van die epoch
print("Beste epoch op basis van mAP@0.5:0.95:")
print(best_epoch_row)

https://docs.ultralytics.com/usage/cfg/?utm_source=chatgpt.com#validation-settings

In [ ]:
# from ultralytics import YOLO
# # Laad het getrainde model (pas pad aan als nodig)
# model = YOLO('YOLOv12sv7.pt').to("cuda") 

# Pad naar je YAML bestand
data_path = r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\YOLOv12v1\data.yaml"

In [ ]:
# ---- mAP op TRAINSET ----
metrics_train = model.val(data=data_path, split='train', save=False, save_json=False, save_txt=False, plots=False)
print("Trainset-resultaten:")
print(f"mAP@0.5: {metrics_train.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics_train.box.map:.4f}")

In [ ]:
# ---- mAP op VALIDATIESET (standaard) ----
metrics_val = model.val(data=data_path, split='val', save=False, save_json=False, save_txt=False, plots=False)  # split is default 'val'
print("Validatie-resultaten:")
print(f"mAP@0.5: {metrics_val.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics_val.box.map:.4f}")